In [1]:
import argparse
import os
import pickle
import pprint
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import os
from utils import *


def set_dates(df, time, mind, maxd, isIndex=False):
    if isIndex:
        newdf = df[(df.index>=mind) & (df.index<=maxd)]
    else:
        newdf = df[(df[time]>=mind) & (df[time]<=maxd)]

    return newdf


data_dir = 'data'
country = pd.read_csv(os.path.join(data_dir, 'COVID_Cases_Restricted_Detailed_12042020.csv'), parse_dates=['cdc_report_dt', 'pos_spec_dt', 'onset_dt'])


print('all cases', country.shape)
percent_missing = country.isnull().sum() * 100 / len(country)
percent_missing
print(country.shape)

country = country[~country['res_state'].isin(['CT','IL','NJ'])]
print(country.shape)

min_date = pd.Timestamp('2020-04-01') - pd.Timedelta(days=6)
max_date = pd.Timestamp('2020-11-01')

country = country[country['current_status']=='Laboratory-confirmed case']


country = set_dates(country, 'cdc_report_dt', min_date, max_date)
percent_missing = country.isnull().sum() * 100 / len(country)
print(percent_missing)

print('----percent missing florida----')
percent_missing = (country[country['res_state']=='FL'].isnull().sum() * 100)/len(country[country['res_state']=='FL'])
print(percent_missing)

print(country.shape)



country['age_group'] = country['age_group'].astype('category')
country['age_group'].cat.categories
country.loc[country['age_group'].isnull(), 'age_group'] = 'Unknown'
country['age_group'].cat.rename_categories({'Unknown': '_' + 'Unknown'}, inplace=True)
country['age_group'].cat.categories

#country['age_group']
country['age_group'].cat.rename_categories({'0 - 9 Years':'0-9', 
                                       '10 - 19 Years':'10-19',
                                       '20 - 29 Years':'20-29',
                                       '30 - 39 Years':'30-39',
                                       '40 - 49 Years':'40-49', 
                                       '50 - 59 Years':'50-59',
                                       '60 - 69 Years':'60-69',
                                       '70 - 79 Years':'70-79',
                                       '80+ Years':'80+'}, inplace=True)





# hosp
country.loc[country['hosp_yn']!='Yes', 'hosp_yn'] = 0
country.loc[country['hosp_yn']=='Yes', 'hosp_yn'] = 1


# death
country.loc[country['death_yn']!='Yes', 'death_yn'] = 0
country.loc[country['death_yn']=='Yes', 'death_yn'] = 1
country = country.rename(columns={'age_group':'Age_group', 
                                    'sex': 'Gender',
                                    'hosp_yn': 'Hospitalized',
                                    'death_yn': 'Died',
                                    'current_status': 'Case'})
country = country.reset_index()


all cases (8405079, 31)
(8405079, 31)
(7370423, 31)
race_ethnicity_combined     3.354641
current_status              0.000000
cdc_report_dt               0.000000
sex                         1.131478
onset_dt                   49.091925
pos_spec_dt                63.518775
hosp_yn                     0.000000
icu_yn                      0.000000
death_yn                    0.000000
hc_work_yn                  3.905486
pna_yn                      0.000000
abxchest_yn                 0.099368
acuterespdistress_yn        0.000000
mechvent_yn                 0.000000
fever_yn                    0.000000
sfever_yn                   0.000000
chills_yn                   0.000000
myalgia_yn                  0.000000
runnose_yn                  0.000000
sthroat_yn                  0.000000
cough_yn                    0.000000
sob_yn                      0.000000
nauseavomit_yn              0.000000
headache_yn                 0.000000
abdom_yn                    0.000000
diarrhea_yn            

In [2]:
DATA_DIR = "../florida/data/"
time = 'Case_'
#print(time)
florida= pd.read_csv(os.path.join(
        DATA_DIR, 'florida_2020-12-04-15-35-01.csv'),index_col=False, parse_dates=[time, "ChartDate"])
florida["Age_group"] = florida["Age_group"].astype('category')
print(florida['Hospitalized'].value_counts())
print(florida['Hospitalized'].unique()) # 'YES'=1, else = 0
print(florida['Died'].unique()) # 'Yes'=1, nan = 0

#florida['Hospitalized'].isna()




florida.loc[florida['Hospitalized'].isna(), 'Hospitalized'] = 'Missing'
florida['Hospitalized'].value_counts()/len(florida)
print(florida.shape)
florida = set_dates(florida, 'Case_', min_date, max_date)
print(florida.shape)


florida['Age_group'] = florida['Age'].apply(lambda x: '0-9' if x < 10 
                                                           else ('10-19' if x < 20 
                                                                else('20-29'if x < 30 
                                                                    else('30-39' if x <40
                                                                        else('40-49' if x < 50
                                                                            else('50-59' if x < 60
                                                                                 else '60-69' if x<70
                                                                                    else '70-79' if x<80
                                                                                        else '80+' if not pd.isnull(x) 
                                                                                             else '_Unknown'))))))

florida['Age_group'] = florida['Age_group'].astype('category')
florida['Age_group'].cat.categories
order_cat = ['0-9',
                                             '10-19',
                                             '20-29',
                                             '30-39',
                                             '40-49',
                                             '50-59',
                                             '60-69',
                                             '70-79',
                                             '80+',
                                             '_Unknown']#[::-1]
florida['Age_group'].cat.reorder_categories(order_cat, inplace=True)
florida = florida.reset_index()


florida.loc[florida['Hospitalized']!='YES', 'Hospitalized'] = 0
florida.loc[florida['Hospitalized']=='YES', 'Hospitalized'] = 1

florida['Died'].fillna(0, inplace=True)
florida.loc[florida['Died']=='Yes', 'Died'] =1 

NO         547028
UNKNOWN    424072
YES         56809
Name: Hospitalized, dtype: int64
['UNKNOWN' 'NO' 'YES' nan]
[nan 'Yes']
(1039207, 15)
(806709, 15)


In [3]:
def reorder_cat(df, var, order):
    #print(df)
    df[var] = df[var].astype('category')
    df[var].cat.reorder_categories(order, inplace=True)
    return df

def rename_cat(df, var, yn=True):
    df[var] = df[var].astype('category')
    names = {1:'Yes', 0:'No'}
    if yn:
        df[var].cat.rename_categories(names, inplace=True)
    #print(df[var].cat.categories)
   # florida['Died'].fillna(0, inplace=True)
    print(df[var].cat.categories)

    return df

florida = rename_cat(florida, 'Hospitalized')
country = rename_cat(country, 'Hospitalized')

florida = rename_cat(florida, 'Died')
country = rename_cat(country, 'Died')
florida = reorder_cat(florida, 'Hospitalized', ['Yes', 'No'])

order_age = ['0-9',
                                             '10-19',
                                             '20-29',
                                             '30-39',
                                             '40-49',
                                             '50-59',
                                             '60-69',
                                             '70-79',
                                             '80+',
                                             '_Unknown']

order_gen = ['Female', 'Male', 'Missing', 'Unknown']

Index(['No', 'Yes'], dtype='object')
Index(['No', 'Yes'], dtype='object')
Index(['No', 'Yes'], dtype='object')
Index(['No', 'Yes'], dtype='object')


In [4]:
def fm(num, ratio):
    return "{} ({:.1f}%)".format(num, 100*ratio)

#def get_cat_count(s, o):
#    return s[o]
# Count
names = ['COVID-19 Cases', 'Age'] + order_age +['Gender']+order_gen+['Hospitalized', 'Yes', 'No','Died', 'Yes', 'No']
fl = []
ct = []
fl.append(len(florida))
ct.append(len(country))

def add_category(florida, country, amt, order, fl, ct):
    # for Hospitalized
    fl_h = florida[amt].value_counts()
    c_h = country[amt].value_counts()

    for o in order:
        #print(fl_h[o])
        fl.append(fm(fl_h[o], fl_h[o]/len(florida)))
        ct.append(fm(c_h[o], c_h[o]/len(country)))
    return fl, ct

def add_category_gender(florida, country, amt, order, fl, ct):
    # for Hospitalized
    fl_h = florida[amt].value_counts()
    c_h = country[amt].value_counts()

    for o in order:
        #print(fl_h[o])        
        if o!='Missing':
            fl.append(fm(fl_h[o], fl_h[o]/len(florida)))
        else:
            fl.append(fm(0, 0))
    for o in order:
            ct.append(fm(c_h[o], c_h[o]/len(country)))
    return fl, ct


fl.append('')
ct.append('')   
fl, ct = add_category(florida, country, 'Age_group', order_age, fl, ct)
fl.append('')
ct.append('')   
fl, ct = add_category_gender(florida, country, 'Gender', order_gen, fl, ct)
fl.append('')
ct.append('')     
fl, ct = add_category(florida, country, 'Hospitalized', ['Yes', 'No'], fl, ct)
fl.append('')
ct.append('')
fl, ct = add_category(florida, country, 'Died', ['Yes', 'No'], fl, ct)

    
    
    
#print(fl)
print(len(names), len(fl), len(ct))
dem = pd.DataFrame({'name':names, 'Florida':fl, 'Country':ct})
dem

dem = dem.set_index('name')
print(dem.to_latex())


23 23 23
\begin{tabular}{lll}
\toprule
{} &         Florida &          Country \\
name           &                 &                  \\
\midrule
COVID-19 Cases &          806709 &          5566170 \\
Age            &                 &                  \\
0-9            &    28763 (3.6\%) &    188885 (3.4\%) \\
10-19          &    72176 (8.9\%) &    540922 (9.7\%) \\
20-29          &  155424 (19.3\%) &  1106791 (19.9\%) \\
30-39          &  138599 (17.2\%) &   918351 (16.5\%) \\
40-49          &  124778 (15.5\%) &   837394 (15.0\%) \\
50-59          &  120581 (14.9\%) &   800695 (14.4\%) \\
60-69          &   81593 (10.1\%) &   558180 (10.0\%) \\
70-79          &    48223 (6.0\%) &    318363 (5.7\%) \\
80+            &    35531 (4.4\%) &    257045 (4.6\%) \\
\_Unknown       &     1041 (0.1\%) &     39544 (0.7\%) \\
Gender         &                 &                  \\
Female         &  415089 (51.5\%) &  2842355 (51.1\%) \\
Male           &  388103 (48.1\%) &  2611573 (46.9\%) \\
Miss